In [1]:
import vlass_search
import csv
import pandas as pd
import re
import os
from urllib.error import HTTPError
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u

C:\Users\Jack Pope\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


File downloaded to: VLASS_dyn_summary.php


C:\Users\Jack Pope\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Jack Pope\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [10]:
transients_df = pd.read_csv("ztf_transients.csv")


In [3]:
def get_png_filenames(transient_name, image_folder='images'):
    """
    Given a transient name in string form, returns all of the png images of the form "{transient_name}_VLASS"
    in a list in the image folder
    """
    
    # Initialize an empty list to store matching filenames
    matching_files = []

    # Iterate over all files in the image folder
    for filename in os.listdir(image_folder):
        # Check if the filename starts with the transient name followed by '_VLASS'
        if filename.startswith(f"{transient_name}_VLASS"):
            # Construct the full path including the 'images/' folder
            full_path = os.path.join(image_folder, filename)
            # Add the full path to the list
            matching_files.append(full_path)

    return matching_files

In [4]:
def extract_flux_values(text):
    # Define regex patterns for peak flux and RMS flux
    peak_flux_pattern = r"Peak Flux (\d+\.\d+)"
    rms_flux_pattern = r"RMS Flux (\d+\.\d+)"

    # Search for the patterns in the text
    peak_flux_match = re.search(peak_flux_pattern, text)
    rms_flux_match = re.search(rms_flux_pattern, text)

    # Extract the values if the patterns are found
    if peak_flux_match and rms_flux_match:
        peak_flux = float(peak_flux_match.group(1))
        rms_flux = float(rms_flux_match.group(1))
        return peak_flux, rms_flux
    else:
        return None, None
    
from PIL import Image
import pytesseract

# Path to the Tesseract-OCR executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def read_flux_from_image(image_path):
    # Open the image file
    img = Image.open(image_path)
    
    # Use pytesseract to do OCR on the image
    text = pytesseract.image_to_string(img)
    
    peak_flux, rms_flux = extract_flux_values(text)
    
    return peak_flux, rms_flux

In [5]:
def extract_epoch(image_name):
    """
    Extract the epoch from the image name.
    
    :param image_name: The name of the image file.
    :return: The extracted epoch or None if not found.
    """
    # Split the string by underscores and take the second part
    try:
        epoch_with_ext = image_name.split('_')[1]
        epoch = epoch_with_ext.split('.png')[0]
        return epoch
    except IndexError:
        return None


In [6]:
def generate_row(name, c, image_dir='images'):
    """
    Generates a row of VLASS PNGs at a given sky coordinate

    Parameters
    -----------
    name: name of the source
    c: coordinates as SkyCoord object
    -----------
    """
    try:
        images, epochs, dates = vlass_search.run_search(name, c)
    except HTTPError as e:
        if e.code == 404:
            print(f"Error 404: URL not found for {name}")
            images, epochs, dates = ['images/unimaged.png'] * 3, ['NA'] * 3, ['Invalid date'] * 3
        else:
            raise e
    
    return {'name': name, 'images': images, 'epochs': epochs, 'vla_dates': dates}

In [15]:
def get_multiple_rows(start, end):

    image_paths = []

    for i in range(start - 1, end):
        name = transients_df['ZTFID'][i]
        ra = transients_df['RA'][i]
        dec = transients_df['Dec'][i]
        ztf_date = transients_df['peakt'][i]
        skyguy = SkyCoord(ra = ra, dec = dec, unit = (u.hourangle, u.deg))
        # add date to dictionary items
        object_dict = generate_row(name, skyguy)
        object_dict['ztf_date'] = ztf_date
        image_paths.append(object_dict)

    for obj in image_paths:
        deltas = []
        print(obj)

        for x in obj['vla_dates']:
            # Convert VLA dates to Julian dates
            if x != 'Invalid date':
                x = Time(x, format = 'iso')
                vla_jd_date = x.jd
            else:
                vla_jd_date = 'NA'
            deltas.append(vla_jd_date)
            print(f"VLA JD Date: {vla_jd_date} (type: {type(vla_jd_date)})")

        obj['ztf_date'] += 2458000
        print(f"ZTF JD Date: {obj['ztf_date']} (type: {type(obj['ztf_date'])})")

        # Ensure the type of ZTF date is float
        ztf_jd_date = float(obj['ztf_date'])

        for i in range(len(deltas)):
            if deltas[i] != 'NA':
                deltas[i] = round(deltas[i] - ztf_jd_date, 2)

        obj['delta_ts'] = deltas
        print(f"Deltas: {deltas}")
        
    return image_paths

get_multiple_rows(1,10)

Running for ZTF17aaapufz
Coordinates <SkyCoord (ICRS): (ra, dec) in deg
    (188.96783333, 27.93205556)>
Date: None

Looking for tile observation for T17t17
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.1v2/T17t17/
Tile Found:
T17t17 VLASS1.1v2
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.1v2/T17t17/VLASS1.1.ql.T17t17.J123340+273000.10.2048.v1/VLASS1.1.ql.T17t17.J123340+273000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
PNG file images\ZTF17aaapufz_VLASS1.1v2.png already exists. Skipping download.
Run search completed in 0.37 seconds.

Looking for tile observation for T17t17
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.1/T17t17/
Tile Found:
T17t17 VLASS2.1
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.1/T17t17/VLASS2.1.ql.T17t17.J123340+273000.10.2048.v1/VLASS2.1.ql.T17t17.J123340+273000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
PNG file images\ZTF17aaapufz_VLASS2.1.png already exists. Skipping download.
Run search completed in 0.31 seconds.

Looking for

[{'name': 'ZTF17aaapufz',
  'images': ['images\\ZTF17aaapufz_VLASS1.1v2.png',
   'images\\ZTF17aaapufz_VLASS2.1.png',
   'images\\ZTF17aaapufz_VLASS3.1.png'],
  'epochs': ['VLASS1.1v2', 'VLASS2.1', 'VLASS3.1'],
  'vla_dates': [<Time object: scale='utc' format='iso' value=2017-11-24 00:00:00.000>,
   <Time object: scale='utc' format='iso' value=2020-09-06 00:00:00.000>,
   <Time object: scale='utc' format='iso' value=2023-02-12 00:00:00.000>],
  'ztf_date': 2459726.75,
  'delta_ts': [-1645.25, -628.25, 260.75]},
 {'name': 'ZTF17aaazdba',
  'images': ['images\\ZTF17aaazdba_VLASS1.2v2.png',
   'images\\ZTF17aaazdba_VLASS2.2.png',
   'images\\unimaged.png'],
  'epochs': ['VLASS1.2v2', 'VLASS2.2', 'VLASS3.2'],
  'vla_dates': [<Time object: scale='utc' format='iso' value=2019-04-14 00:00:00.000>,
   <Time object: scale='utc' format='iso' value=2021-12-13 00:00:00.000>,
   <Time object: scale='utc' format='iso' value=2024-05-04 00:00:00.000>],
  'ztf_date': 2458561.73,
  'delta_ts': [25.77, 9

In [7]:
final_transients = []



test_ra = "14:28:15.92"
test_dec = "-01:58:55.7"
test_object = SkyCoord(ra = test_ra, dec = test_dec, unit=(u.hourangle, u.deg))

vlass_search.run_search("aa", test_object)




Running for aa
Coordinates <SkyCoord (ICRS): (ra, dec) in deg
    (217.06633333, -1.98213889)>
Date: None

Looking for tile observation for T10t22
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.2v2/T10t22/
Tile Found:
T10t22 VLASS1.2v2
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.2v2/T10t22/VLASS1.2.ql.T10t22.J143000-013000.10.2048.v1/VLASS1.2.ql.T10t22.J143000-013000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
curl -o new_fits\VLASS1.2.ql.T10t22.J143000-013000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits https://archive-new.nrao.edu/vlass/quicklook/VLASS1.2v2/T10t22/VLASS1.2.ql.T10t22.J143000-013000.10.2048.v1/VLASS1.2.ql.T10t22.J143000-013000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
Generating cutout
Cutout centered at position 217.06633333333332, -1.9821388888888891


Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


PNG saved successfully: images\aa_VLASS1.2v2.png
Sorry, tile has not been imaged at the position of the source
Run search completed in 1.75 seconds.

Looking for tile observation for T10t22
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.2/T10t22/
Tile Found:
T10t22 VLASS2.2
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.2/T10t22/VLASS2.2.ql.T10t22.J143000-013000.10.2048.v1/VLASS2.2.ql.T10t22.J143000-013000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
curl -o new_fits\VLASS2.2.ql.T10t22.J143000-013000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits https://archive-new.nrao.edu/vlass/quicklook/VLASS2.2/T10t22/VLASS2.2.ql.T10t22.J143000-013000.10.2048.v1/VLASS2.2.ql.T10t22.J143000-013000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
Generating cutout
Cutout centered at position 217.06633333333332, -1.9821388888888891


Set MJD-END to 59511.824422 from DATE-END'. [astropy.wcs.wcs]


PNG saved successfully: images\aa_VLASS2.2.png
Sorry, tile has not been imaged at the position of the source
Run search completed in 2.13 seconds.

Looking for tile observation for T10t22
Sorry, tile will be observed later in this epoch


(['images\\unimaged.png', 'images\\unimaged.png', 'images\\unimaged.png'],
 ['VLASS1.2v2', 'VLASS2.2', 'VLASS3.2'],
 [<Time object: scale='utc' format='iso' value=2019-05-01 00:00:00.000>,
  <Time object: scale='utc' format='iso' value=2021-10-24 00:00:00.000>,
  <Time object: scale='utc' format='iso' value=2024-06-14 00:00:00.000>])

In [ ]:
# def create_csv(transients, csv_filename = "interesting_transients.csv"):

#     with open(csv_filename, mode='w', newline='') as file:
        
#         writer = csv.writer(file) 
#         # Write the header
#         writer.writerow(["Transient Name", "Epoch", "Peak Flux", "RMS Flux", "Delta T"])

#         # Loop through the list of transients
#         for i in range(len(transients)):
#             transient_name = transients['ZTFID'][i]
#             image_names = get_png_filenames(transient_name)
#             object_dict = get_multiple_rows(i + 1, i + 1)[0]
#             for image in image_names:
#                 peak_flux, rms_flux = read_flux_from_image(image)
#                 epoch = extract_epoch(image)
#                 for ii in range(len(object_dict['images'])):
#                     if object_dict['images'][ii] == image:
#                         delta_t = object_dict['delta_ts'][ii]
#                 writer.writerow([transient_name, epoch, peak_flux, rms_flux, delta_t])

# create_csv(transients_df)

In [13]:
import os
import csv

def create_csv(transients, csv_filename="interesting_transients.csv", start=0, end=None):
    # Check if the file already exists
    file_exists = os.path.isfile(csv_filename)
    
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header only if the file doesn't exist
        if not file_exists:
            writer.writerow(["Transient Name", "Epoch", "Peak Flux", "RMS Flux", "Delta T"])

        # Determine the range of objects to process
        if end is None:
            end = len(transients)

        # Loop through the specified range of transients
        for i in range(start, end + 1):
            transient_name = transients['ZTFID'][i]
            image_names = get_png_filenames(transient_name)
            object_dict = get_multiple_rows(i + 1, i + 1)[0]
            for image in image_names:
                peak_flux, rms_flux = read_flux_from_image(image)
                epoch = extract_epoch(image)
                for ii in range(len(object_dict['images'])):
                    if object_dict['images'][ii] == image:
                        delta_t = object_dict['delta_ts'][ii]
                writer.writerow([transient_name, epoch, peak_flux, rms_flux, delta_t])


create_csv(transients_df, "interesting_transients.csv", start = 8776, end = 8851)


Running for ZTF24aaekbxi
Coordinates <SkyCoord (ICRS): (ra, dec) in deg
    (247.153625, 40.93391667)>
Date: None

Looking for tile observation for T21t17
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.2v2/T21t17/
Tile Found:
T21t17 VLASS1.2v2
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.2v2/T21t17/VLASS1.2.ql.T21t17.J162843+403000.10.2048.v1/VLASS1.2.ql.T21t17.J162843+403000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
PNG file images\ZTF24aaekbxi_VLASS1.2v2.png already exists. Skipping download.
Run search completed in 1.97 seconds.

Looking for tile observation for T21t17
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.2/T21t17/
Tile Found:
T21t17 VLASS2.2
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.2/T21t17/VLASS2.2.ql.T21t17.J162843+403000.10.2048.v1/VLASS2.2.ql.T21t17.J162843+403000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
PNG file images\ZTF24aaekbxi_VLASS2.2.png already exists. Skipping download.
Run search completed in 2.13 seconds.

Looking for t

KeyError: 8851

In [13]:
import importlib
importlib.reload(vlass_search)

File downloaded to: VLASS_dyn_summary.php


<module 'vlass_search' from 'd:\\summer_research_code\\Query_VLASS_2\\vlass_search.py'>